In [1]:
%load_ext autoreload
%autoreload 2

Based on 



1.   https://towardsdatascience.com/hugging-face-transformers-fine-tuning-distilbert-for-binary-classification-tasks-490f1d192379
2.   https://www.analyticsvidhya.com/blog/2020/07/transfer-learning-for-nlp-fine-tuning-bert-for-text-classification/
3.   https://huggingface.co/transformers/training.html#fine-tuning-with-keras




**problem statement:**


*   a developer has to inspect an **artifact X**
*   Within the artifact, only a portion of the text is relevant to **input task Y**
*   We ought to build a model that establishes relationships between **Y** and **sentences x ∈ X** 
*  The model must determine: **is x relevant to task Y**




<br>

___

*Example of a task and an annotated artifact:*

<br>

[<img src="https://i.imgur.com/Zj1317H.jpg">](https://i.imgur.com/Zj1317H.jpg)




* The coloured sentences are sentences annotated as relevant to the input task. 
* The warmer the color, the more annotators selected that portion of the text. 
* For simplicity, we process the data and used sentences 

<br>

___

*Ultimately, our data is a tuple representing:*


*   **text** = artifact sentence

*   **question** = task description

*   **source** = URL of the artifact

*   **category_index** = whether sentence is relevant [or not] for the input task

*   **weights** = number of participants who annotated sentence as relevant


<br>

___



In [2]:
# @title Import data as JSON
import itertools
import json
import logging
import os
import sys
import random
from pathlib import Path

from Levenshtein import ratio
from colorama import Fore, Style

logger = logging.getLogger()
logger.level = logging.DEBUG
stream_handler = logging.StreamHandler(sys.stdout)
logger.addHandler(stream_handler)

from ds_android import get_input_for_BERT

raw_data = get_input_for_BERT()

print('Sample entry from data:')
print(json.dumps(raw_data[0], indent=4, sort_keys=True))

39 129  https://developer.android.com/training/permissions/requesting
14 21  https://stackoverflow.com/questions/5233543
4 34  https://github.com/morenoh149/react-native-contacts/issues/516
27 63  https://guides.codepath.com/android/Understanding-App-Permissions
9 161  https://www.avg.com/en/signal/guide-to-android-app-permissions-how-to-use-them-smartly
9 15  https://developer.android.com/training/volley/request
14 65  https://stackoverflow.com/questions/28504524
20 59  https://medium.com/@JasonCromer/android-asynctask-http-request-tutorial-6b429d833e28
5 97  https://www.twilio.com/blog/5-ways-to-make-http-requests-in-java
4 12  https://stackoverflow.com/questions/33241952
6 33  https://github.com/realm/realm-java/issues/776
3 17  https://stackoverflow.com/questions/8712652
8 59  https://dzone.com/articles/android-rotate-and-scale
5 470  https://developer.android.com/reference/android/widget/TextView
7 11  https://stackoverflow.com/questions/19025301
8 95  https://docs.oracle.com/java

4 54  https://developer.android.com/training/gestures/scroll
4 16  https://stackoverflow.com/questions/39588322
20 196  https://developer.android.com/training/dependency-injection/dagger-android
6 44  https://stackoverflow.com/questions/57235136
24 121  https://guides.codepath.com/android/dependency-injection-with-dagger-2
Sample entry from data:
{
    "category_index": 1,
    "question": "Permission Denial when trying to access contacts in Android",
    "source": "https://developer.android.com/training/permissions/requesting",
    "text": "Every Android app runs in a limited-access sandbox.",
    "weights": 1
}


In [3]:
# @title DICT that will store fold results
# If there is a previous execution for the same configuration, we load it from disk

# final results are the average of 3 distinct runs of this script.
# reason: avoid phishing results when BERT training procedures were exceptionally good
NUMBER_OF_RUNS = 3 
# config_output = 'output/bert_ds_android_base.json'
config_output = 'output/bert_ds_android_fe.json' # for frame-elements filter
# config_output = 'output/bert_ds_android_fa.json' # for frame-association filters

fold_results = dict()

if os.path.isfile(config_output):
    logger.info(Fore.YELLOW + "Loading data from cache" + Style.RESET_ALL)
    with open(config_output) as input_file:
        fold_results = json.load(input_file)
        
if 'venn_diagram_set' not in fold_results:
    fold_results['venn_diagram_set'] = []     

Loading data from cache


In [4]:
# @title Set environment variables

import os
import contextlib
import tensorflow as tf
import os
import codecs
import numpy as np
import math
import json

import numpy as np
import pandas as pd

from collections import defaultdict, Counter
from tqdm import tqdm

USE_TPU = False
os.environ['TF_KERAS'] = '1'

# @title Initialize TPU Strategy
if USE_TPU:
    TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)
    tf.contrib.distribute.initialize_tpu_system(resolver)
    strategy = tf.contrib.distribute.TPUStrategy(resolver)
    
from TFBertForTaskTextClassification import TFBertForTaskTextClassification
from TFBertForTaskTextClassification import TFBertForAndroidTaskTextClassification
from TFBertForTaskTextClassification import TFBertForSyntheticTaskTextClassification 

from metrics import MetricsAggregator

Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.


# BERT

In [5]:
# Init the model
model = TFBertForAndroidTaskTextClassification(model_id = 'bert-base-uncased')

# Configure filters. All other values are as default
model.target_output = 10
model.use_frame_filtering = False
model.match_frame_from_task = False
model.n_undersampling = 3
        
# Load tokenizer
model.tokenizer(cache_dir='/home/msarthur/scratch', local_files_only=True)

In [6]:
# base + frame-elements
model.use_frame_filtering = True
model.match_frame_from_task = False

In [7]:
# # base + frame-associations
# model.use_frame_filtering = False
# model.match_frame_from_task = True

In [8]:
# @title 10-fold cross validation WIP
CORPUS = raw_data

all_tasks = sorted(list(set([d['question'] for d in raw_data])))
rseed = 20210343
random.seed(rseed)
random.shuffle(all_tasks)

from sklearn.model_selection import KFold


file_handler = logging.FileHandler('/home/msarthur/scratch/LOG-bert_ds_android.ans')
file_handler.setLevel(logging.DEBUG)
logger.addHandler(file_handler)


n_splits = 10
kf = KFold(n_splits=n_splits, random_state=rseed)
np_tasks_arr = np.array(all_tasks)


idx_split = 0
for train_index, test_index in kf.split(np_tasks_arr):

    idx_split = str(idx_split)
    eval_fold = True
    # 10 runs per fold to avoid reporting peek results in a given fold
    if idx_split in fold_results and fold_results[idx_split]['run_cnt'] >= NUMBER_OF_RUNS:
        logger.info(Fore.RED + f"Fold {idx_split} FULLY TESTED" + Style.RESET_ALL)
        eval_fold = False


    if eval_fold:
        model.metrics.reset_aggregators()

        test_tasks_lst = np_tasks_arr[test_index].tolist()

        logger.info("")
        logger.info(Fore.RED + f"Fold {idx_split}" + Style.RESET_ALL)
        logger.info('\n'.join(test_tasks_lst))

        # <------------------------------------------------------------------------- INPUT
        df_train, df_val, df_test, weights = model.get_train_val_test(
            CORPUS, test_tasks_lst
        )
        

        logger.info('-' * 10)
        logger.info(Fore.RED + 'train'+ Style.RESET_ALL)
        logger.info(str(df_train.category_index.value_counts()))
        logger.info("")

        logger.info(Fore.RED + 'test'+ Style.RESET_ALL)
        logger.info(str(df_test.category_index.value_counts()))
        logger.info("")

        logger.info(Fore.RED + 'weights'+ Style.RESET_ALL)
        logger.info(str(weights))
        logger.info('-' * 10)
        
        # <------------------------------------------------------------------------- TRAIN

        # Encode X_train
        train_encodings = model.encode(df_train)
        train_labels = df_train['category_index'].tolist()

        # Encode X_valid
        val_encodings = model.encode(df_val)
        val_labels = df_val['category_index'].tolist()


        # https://huggingface.co/transformers/custom_datasets.html
        train_dataset = tf.data.Dataset.from_tensor_slices((
            dict(train_encodings),
            train_labels
        ))

        val_dataset = tf.data.Dataset.from_tensor_slices((
            dict(val_encodings),
            val_labels
        ))


        fine_tunned_keras_model = model.build(
            train_dataset, val_dataset, weights, 
            checkpoint_filepath='/home/msarthur/scratch/best_model', 
            cache_dir='/home/msarthur/scratch', 
            local_files_only=True
        )
        
        
        if model.match_frame_from_task:
            __frame_pairs = model.fn_frame_pairs.get_most_common_frame_relationships(df_train)
            model.sentence_task_frame_pairs = __frame_pairs

        # <------------------------------------------------------------------------- TEST
        logger.info("")
        logger.info(Fore.RED + f"Testing model" + Style.RESET_ALL)
        for source in df_test["source"].unique():
            df_source = df_test[df_test["source"] == source]   
            logger.info(source)
            
            model.test(source, df_source, fine_tunned_keras_model)
                        

        # <------------------------------------------------------------------------- METRICS   
        
        prediction_metrics, api_metrics, so_metrics, git_metrics, misc_metrics = model.get_evaluation_metrics()
        
        MetricsAggregator.add_idx_fold_results(
            idx_split, fold_results, prediction_metrics,
            api_metrics, so_metrics, git_metrics, misc_metrics
        )

        fold_results['venn_diagram_set'] += model.metrics.venn_diagram_set
        fold_results['venn_diagram_set'] = list(set(fold_results['venn_diagram_set']))


        _precision, _recall, _f1score = MetricsAggregator.avg_macro_metric_for(prediction_metrics)

        logger.info("")
        logger.info(Fore.YELLOW + "Model metrics" + Style.RESET_ALL)
        logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
        logger.info("recall:    " + Fore.RED + "{:.3f}".format(_recall) + Style.RESET_ALL)
        logger.info("f1-score:  " + Fore.RED + "{:.3f}".format(_f1score) + Style.RESET_ALL)


    idx_split = int(idx_split)
    idx_split += 1
    logger.info(f"next {idx_split}")
    


Fold 0
how can i get the value of text view in recyclerview item?
Hide MarkerView when nothing selected
How to check programmatically whether app is running in debug mode or not?
JSONObject parse dictionary objects
Want to add drawable icons insteadof colorful dots


/home/msarthur/hface/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
100%|██████████| 7908/7908 [00:00<00:00, 826857.36it/s]

----------
train
0    742
1    248
Name: category_index, dtype: int64

test
0    706
1     29
Name: category_index, dtype: int64

weights
{0: 1.0, 1: 3.0}
----------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
AutoGraph could not transform <bound method Socket.send of <zmq.sugar.socket.Socket object at 0x2b57a025c3d0>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expe

...

/home/msarthur/hface/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/msarthur/hface/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 7908/7908 [00:00<00:00, 872604.14it/s]

----------
train
0    753
1    251
Name: category_index, dtype: int64

test
0    553
1     25
Name: category_index, dtype: int64

weights
{0: 1.0, 1: 3.0}
----------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
16/16 [==============================] - ETA: 0s - loss: 1.0667 - sparse_categorical_accuracy: 0.5827The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output

/home/msarthur/hface/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/msarthur/hface/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [9]:
# with open(config_output, 'w') as outfile:
#     json.dump(fold_results, outfile, sort_keys=True, indent=4)
#     logger.info(Fore.RED + "Output successfully saved to: {}".format(config_output) + Style.RESET_ALL)

## Results

In [10]:
__precision, __recall, __fscore = MetricsAggregator.get_full_exec_results(fold_results)

logger.info("\n")
logger.info(Fore.RED + "AGGREGATED METRICS" + Style.RESET_ALL)
logger.info("\nprecision: " + Fore.RED + "{:.3f}".format(np.mean(__precision)) + Style.RESET_ALL)
logger.info("recall:    " + Fore.RED + "{:.3f}".format(np.mean(__recall)) + Style.RESET_ALL)
logger.info("f1-score:  " +  Fore.RED + "{:.3f}".format(np.mean(__fscore)) + Style.RESET_ALL)

if np.mean(__precision) >= 0.54:
    with open(config_output, 'w') as outfile:
        json.dump(fold_results, outfile, sort_keys=True, indent=4)
        logger.info(Fore.RED + "Output successfully saved to: {}".format(config_output) + Style.RESET_ALL)



AGGREGATED METRICS

precision: 0.541
recall:    0.591
f1-score:  0.521
Output successfully saved to: output/bert_ds_android_fe.json


## Stack Overflow results

In [11]:
__precision, __recall, __fscore = MetricsAggregator.get_full_exec_results(fold_results, result_type="so")

logger.info("\n")
logger.info(Fore.RED + "Stack Overflow metrics" + Style.RESET_ALL)
logger.info("\nprecision: " + Fore.RED + "{:.3f}".format(np.mean(__precision)) + Style.RESET_ALL)
logger.info("recall:    " + Fore.RED + "{:.3f}".format(np.mean(__recall)) + Style.RESET_ALL)
logger.info("f1-score:  " +  Fore.RED + "{:.3f}".format(np.mean(__fscore)) + Style.RESET_ALL)




Stack Overflow metrics

precision: 0.574
recall:    0.637
f1-score:  0.552


## Github issues results

In [12]:
__precision, __recall, __fscore = MetricsAggregator.get_full_exec_results(fold_results, result_type="git")

logger.info("\n")
logger.info(Fore.RED + "Github issues metrics" + Style.RESET_ALL)
logger.info("\nprecision: " + Fore.RED + "{:.3f}".format(np.mean(__precision)) + Style.RESET_ALL)
logger.info("recall:    " + Fore.RED + "{:.3f}".format(np.mean(__recall)) + Style.RESET_ALL)
logger.info("f1-score:  " +  Fore.RED + "{:.3f}".format(np.mean(__fscore)) + Style.RESET_ALL)



Github issues metrics

precision: 0.519
recall:    0.552
f1-score:  0.488


## Examples of text retrieved

In [13]:
#@title Sample prediction outputs for API sources

logger.info(Fore.RED + "API" + Style.RESET_ALL)
model.metrics.examples_per_source_type(source_type='api', n_samples=8)

API


In [14]:
#@title Sample prediction outputs for GIT sources

logger.info(Fore.RED + "GIT" + Style.RESET_ALL)
model.metrics.examples_per_source_type(source_type='git', n_samples=8)

GIT


In [15]:
#@title Sample prediction outputs for SO sources

logger.info(Fore.RED + "SO" + Style.RESET_ALL)
model.metrics.examples_per_source_type(source_type='so', n_samples=8)

SO

How to Integrate reCAPTCHA 2.0 in Android
https://stackoverflow.com/questions/27297067

[w=0][y=1][p=0.6527] Hi curious you can validate your google recaptcha at client side also 100 % work for me to verify your google recaptcha just see below code This code at the html body:

[w=3][y=1][p=0.6478] The cool thing about the new Google Recaptcha is that the validation is now completely encapsulated in the widget.

[w=3][y=1][p=0.6434] And replace the response_string with the value that you earlier got by the g-recaptcha-response field.

[w=0][y=1][p=0.2746] More information here: LINK

[w=0][y=1][p=0.2283] LINK

[w=2][y=1][p=0.2034] Validating the response is really easy.

[w=1][y=1][p=0.1590] So you have to validate this token.

[w=3][y=1][p=0.1586] Just make a GET Request to

--------------------

Application icon doesn&#39;t show up in Android action bar
https://stackoverflow.com/questions/26838730

[w=0][y=1][p=0.8622] Attribute logo is only used in API level 11 and higher -LRB- c

In [16]:
#@title Sample prediction outputs for MISC sources

logger.info(Fore.RED + "MISC" + Style.RESET_ALL)
model.metrics.examples_per_source_type(source_type='misc', n_samples=8)

MISC


In [17]:
logger.info(Fore.RED + f"{len(fold_results['venn_diagram_set'])} entries VENN SET" + Style.RESET_ALL)
for _t in fold_results['venn_diagram_set']:
    logger.info(_t)

267 entries VENN SET

`` It also gives the user more control over the app's functionality ; for example, a user could choose to give a camera app access to the camera but not to the device location.''
Also the focus is on MockWebServer and I wont be explaining Dagger.Enough talking !
And finally:
Now with the adapter in place, we can also easily access any fragments within the ViewPager using getRegisteredFragment:
gradle
Here was my approach to this in kotlin
Then in our `` onStart'' method we call the `` connect'' method and wait for `` onConnected'' callback method be invoked:
Use this to benchmark any gains as you optimize.
Download the source locally and import as a android using android-studios.
Get news and tips by email Subscribe
Hi curious you can validate your google recaptcha at client side also 100 % work for me to verify your google recaptcha just see below code This code at the html body:
RemoteControlClient was what you were looking for, but now it's deprecated and has b

Since the version of Retrofit 2.6.0, you can get the annotations in OkHttp Interceptor using the tag field like this:
I made code for imageview with pinch to zoom using zoomageview.
Let's these step-by-step with an example.How to use PdfRenderer?Initially, let's check basic steps before going through the codeStep1: Get a seekable file descriptor from our pdf document: Step 2: Now let's create the PDFRenderer instance using the ParcleFileDescriptor obtained above: Step 3: Let's create the Bitmap instance with the required dimensions: Step 4: Not get the page to render using PdfRenderer.Page by just passing the index of the pageStep 5: Lastly, render the page to bitmap created in step 3: Step 6: Close the page and renderer once you are doneExampleLet's check how to open the pdf document in fragment providing Uri as an argument to it.
The proposed solutions are correct but to achieve the result you need to set the initial value of your viewpager to Integer.MAX _ VALUE/2.
A reference to th

EXAMPLE:
In GsonCacheableConverter, it overrides responseBodyConverter -LRB- -RRB- to persist response tagged with @Cacheable.
Use ANALYZE to allow SQLite's query planner to work more efficiently.
It uses a custom CallAdapter to get annotation @Authenticated, and put data into a map, which later parsed in the Interceptor.
I think requestBodyConverter -LRB- -RRB- and CallAdapter are closer to your requirement.
Content and code samples on this page are subject to the licenses described in the Content License.
You can download the final project for this tutorial here.
For this guide, we will be using the Yelp API as the example.
use
you need to override getCount method of the BaseAdapter to return total number of views to diplay.
The API call might be executed in an Activity now when the user performs a search.
I hope someone will find this useful.
The more indexes you have, the slower your INSERTs will be, so you will have to work out the best trade-off for your situation.
For example, t